#Reinforcement Learning - Tic Tac Toe
##Brady Young

##Modules

In [0]:
import numpy as np
import copy
import random
import math
import pickle

##Tic Tac Toe Game

In [0]:
class TicTacToe:
  def __init__(self, width=3, state=None):
    #print("Creating board of size: ", str(width**2))
    self.width = width
    self.winner = None
    self.done = False
    self.rewards = { ' ':    2.0,  ## Draw
                     'O':   10.0,  ## Win
                     'X':  -30.0,  ## Lose
                     None:   -1.0}  ## Time Step
    
    if state is None:
      self.state = [[' ' for x in range(self.width)] for x in range(self.width)]
    else:
      self.state = copy.deepcopy(state)
    
  def __repr__(self):
    for x in self.state:
      print(x)
    return ''
  
  def play_match(self, agent, opponent):
    self.state = [[' ' for x in range(self.width)] for x in range(self.width)]
    
    self.agent_mark = agent.mark
    self.opp_mark = opponent.mark
    
    players = random.sample((agent, opponent), k=2)
    
    ## While the game is not over
    while not self.done:
      prev_state = self.state
      self.state = self.place_mark(players[0].choose_action(), players[0].mark)
      agent.learn(prev_state, self.state)
      self.is_done()
      players.reverse()
    return self.winner
  
    
  def encode_state(self, state):
    encoding = 0
    values = { 'O':     1,
               'X':  2}
    w = self.width
      
    for i in range(w):
      for j in range(w):
        mark = state[i][j]
        if mark is not ' ':
          encoding += values[mark] * (3 ** ((i * w) + j))
          
    return int(encoding)
  
  def get_empty(self):
    empty = []
    for i in range(self.width):
      for j in range(self.width):
        if self.state[i][j] is ' ':
          empty.append((i, j))
    return empty
  
  
  def is_full(self):
    for cell in self.state:
      for value in cell:
        if value is ' ':
          return False    
    return True
  
  
  def place_mark(self, pos, mark):
    state = copy.deepcopy(self.state)
    state[pos[0]][pos[1]] = mark
    return state
  
  def is_done(self):
    if not self.done:
      if self.winner is None:
        winner = self.find_vertical()
        if winner is None:
          winner = self.find_horizontal()
          if winner is None:
            winner = self.find_diagonal()
            if winner is None:
              if self.is_full() is True:
                self.winner = ' '
                self.done = True
                return True
              else:
                return False
        self.winner = winner
        self.done = True
    return True
  
  def get_reward(self, state):
    temp = copy.deepcopy(self.state)
    temp_win = self.winner
    temp_done = self.done
    
    self.state = state
    if not self.done:
      self.is_done()
      self.done = temp_done
      
    reward = self.rewards[self.winner]
    self.state = temp
    self.winner = temp_win
    return reward
  
  
  def find_vertical(self):
    state = self.state
    for i in range(self.width):
      if state[0][i] is not ' ':
        mark = state[0][i]
        
        for j in range(self.width):
          if state[j][i] is not mark:
            break
          if j == self.width-1:
            return mark
          
    return None
  
  
  def find_horizontal(self):
    state = self.state 
    for i in range(self.width):
      if state[i][0] is not ' ':
        mark = state[i][0]
        
        for j in range(self.width):
          if state[i][j] is not mark:
            break
          if j == self.width-1:
            return mark
          
    return None
  
  
  def find_diagonal(self):
    state = self.state
    if state[0][0] is not ' ':
      mark = state[0][0]
      for i in range(self.width):
        if state[i][i] is not mark:
          break
        if i == (self.width-1):
          return mark
      
    if state[0][self.width-1] is not ' ':
      mark = state[0][self.width-1]
      for i in range(self.width):
        if state[i][self.width-1-i] is not mark:
          break
        if i == (self.width-1):
          return mark
    
    return None

## Player / Agent

In [0]:
class Player:
  def __init__(self, board, mark):
    self.board = board
    self.mark = mark
    self.stats = { 'won' :  0,
                   'lost':  0,
                   'draw':  0,
                   'total': 0}
    
  def __repr__(self):
    out = "---Agent---" + "\nGames Won:\t" + str(self.stats['won']) + \
        "\nGames Lost:\t"+ str(self.stats['lost'])+ \
        "\nGames Draw:\t"+ str(self.stats['draw'])+ \
        "\nWinning Percentage: " + str(self.stats['won'] / self.stats['total'])

    return out + '\n'
  
  def reset_stats(self):
    for s in self.stats:
      s = 0
      
  def choose_action(self):
    empty = self.board.get_empty()
    return random.choice(empty)

class Human(Player):
  def __init__(self, board, mark):
    super(Human, self).__init__(board, mark)
    
  def choose_action(self):
    print(self.board)
    choice = None
    
    while choice is None:
      choice = input("Choose a position as: x,y ").split(',')
      choice = (int(choice[1]), int(choice[0]))
      if choice not in self.board.get_empty():
        print("Position ", str(choice) , "is occupied.")
        choice = None
        
    return choice
        
class Agent(Player):
  def __init__(self, board, mark, epsilon=0.9, gamma=0.75):
    super(Agent, self).__init__(board, mark)

    self.epsilon = epsilon      ## Chance to use random action
    self.gamma = gamma          ## Reward discount
    self.state_memory = {}
    self.total_games = 0
      
  def save(self):
    with open('agent.pickle', mode='rb') as file:
      file = pickle(self)
  

      
  def learn(self, state, state_d):
    state_key = self.board.encode_state(state)
    state_d_key = self.board.encode_state(state_d)

    self.add_state(state)
    count, reward = self.state_memory[state_key]
    
    count += 1;
    alpha = 1 / count;
    
    self.add_state(state_d)
    count_d, reward_d = self.state_memory[state_d_key]
    
    if self.board.done is True:
      reward = reward + (alpha * (self.board.get_reward(state) - reward))
      self.state_memory[state_key] = (count, reward)
      return 
    
    reward = reward + alpha * (-1 + (self.gamma * reward_d) - reward)
    self.state_memory[state_key] = (count, reward)
    return

  
  def add_state(self, state):
    s = self.board.encode_state(state)
    if s not in self.state_memory:
      self.state_memory[s] = (0, self.board.get_reward(state))

    
  def choose_action(self):
    rand = np.random.random()
    possible_moves = self.board.get_empty()

    if rand > self.epsilon:
      return random.choice(possible_moves)
    
    possible_rewards = []
    for p_m in possible_moves:
      state_d = self.board.place_mark(p_m, self.mark)
      encoded = self.board.encode_state(state_d)
      self.add_state(state_d)
      possible_rewards.append(self.state_memory[encoded][1])
    return possible_moves[possible_rewards.index(max(possible_rewards))]
    

In [0]:
## The recursive training function
## 
## Runs through two separate training regiments,
## decreasing the chance for random action choice
## on each successive game
##
## Last game of each regiment has no random choices
##
class Recurser:
  def __init__(self, agent, opponent):
    self.count = 0
    self.x_winner = True
    initial_epsilon = agent.epsilon
    
    
    ## Train with agent playing first
    while self.x_winner:
      self.x_winner = False
      game = TicTacToe()
      self.recurse(game, agent, opponent)
      if agent.epsilon < 1.0: agent.epsilon += 0.0005
    print("Finished training with agent starting.")
    
    ## Train with opponent playing first
    agent.epsilon = initial_epsilon
    self.x_winner = True
    while self.x_winner:
      self.x_winner = False
      game = TicTacToe()
      self.recurse(game, opponent, agent)
      if agent.epsilon < 1.0: agent.epsilon += 0.0005
    print("Finished training with opponent starting.")

    
  ## Main training loop
  def recurse(self, game, player_one, player_two):
    if game.is_done():
      self.count += 1
      return game.winner

    ## If agent is player, choose an action based on policy
    if type(player_one) is Agent:
      # Engage loop by setting winner to X
      winner = 'X'
      while winner is 'X':
        game_d = TicTacToe(state=game.state)
        player_one.board = game_d
        move = player_one.choose_action()
        state_d = game_d.place_mark(move, player_one.mark) 
        player_one.learn(game.state, state_d)
        game_d.state = state_d
        winner = self.recurse(game_d, player_two, player_one)
      return
    
    ## If opponent is player, iterate through possible states
    ## until agent win or draw is found
    for move in game.get_empty():
      game_d = TicTacToe(state=game.state)
      player_two.board = game_d

      game_d.state = game_d.place_mark(move, player_one.mark)
      player_two.learn(game.state, game_d.state)
      winner = self.recurse(game_d, player_two, player_one)
      if winner is 'X':
        self.x_winner = True
      elif winner in ('O', ' '):
        return

      
## Wrapper for the recursive call
def exaustive_train_agent(agent):
  print("Beginning training...")
  r = Recurser(agent, Player(game, 'X'))
  print("Training complete.")


## Test the agent against an opponent, tracking games played
def test_agent(agent, opponent, n_games):
  test_total = 0
  while test_total < n_games:
    game = TicTacToe()
    agent.board = game
    opponent.board = game
    winner = game.play_match(agent, opponent)
    if winner is agent.mark:
      agent.stats['won'] += 1
    elif winner is opponent.mark:
      agent.stats['lost']+= 1
    else:
      agent.stats['draw'] += 1

    agent.stats['total'] += 1

    test_total += 1
    
    
  print(agent)
  agent.reset_stats()
  
  if type(opponent) is Human:
    print(game)
    print(game.winner + " is Winner!")

## Execution

## Training

In [0]:
game = TicTacToe()
epsilon = 0.5
gamma = 0.9
agent = Agent(game, 'O', epsilon, gamma)

exaustive_train_agent(agent)

## AI vs Random-Choice

In [0]:
opponent = Player(game, 'X')
test_agent(agent, opponent, 1000)

## AI vs Human

In [0]:
opponent = Human(game, 'X')
test_agent(agent, opponent, 1)